In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
!pip install dicom
!pip install pydicom
import dicom
import scipy.ndimage
from __future__ import division
import pydicom
from io import BytesIO
import google.datalab.storage as storage
import matplotlib.pyplot as plt
from skimage import filters
from scipy.ndimage.interpolation import rotate
import math
import numpy as np
from sklearn.metrics import accuracy_score
!pip install keras
!pip install h5py
import keras 
from keras import layers 
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization,Conv2D
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from scipy.ndimage.interpolation import zoom
%gcs read --o "gs://msadata/stage1_labels.csv/stage1_labels.csv" --variable df
df=pd.read_csv(BytesIO(df))
#y=np.array([int(x) for x in df['cancer'].values])
df=df.sort_values(['cancer'])
total=list()
train=list()
valid=list()
test=list()
for o in storage.Bucket('msadata').objects(prefix='test2',delimiter=''):
    total.append(str(o.key).split('/')[1])
total=list(set(total))
total=sorted(total)
y=np.array(df['cancer'])
total=pd.DataFrame(total,columns=['id'])
total=total.merge(df,how='left',on='id')   
total_na=total.isnull().sum()
total=total.fillna(0)
if total.shape[1]>2:
    total.drop(columns=[x for x in t.columns[2:]],inplace=True)
for x in range(2,10):
  if len(total)%x==0:
    ratio=1/x
    break
  else:
    pass
for x in range(2,10):
  if len(total)%x==0:
    ratio=1/x
    break
  else:
    pass
train,test=train_test_split(total.iloc[1:,:],test_size=ratio)
for x in range(2,10):
  if len(train)%x==0:
    ratio=1/x
    break
  else:
    pass
valid,test=train_test_split(test,test_size=ratio)
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)
valid.reset_index(drop=True,inplace=True)

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/usr/local/envs/py3env/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train.reset_index(drop=True,inplace=True)
test.reset_index(drop=True,inplace=True)
valid.reset_index(drop=True,inplace=True)

In [3]:
from keras.layers import Flatten,Dense
from keras.utils import to_categorical
repeat='notfoo'
if repeat=='foo':
  model = load_model('my_model.h5')
else:
  model=Sequential()
  model.add(Conv2D(80,(10),padding='same',activation='relu',data_format='channels_first',input_shape=(150,512,512)))
  model.add(Conv2D(50,(10),padding='same',activation='relu',data_format='channels_first'))
  model.add(Conv2D(1,(1),padding='same',activation='relu',data_format='channels_first'))
  model.add(Flatten())
  model.add(Dense(2,activation='relu'))
  model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
  i=0     


In [7]:

def array_generator(train):
  if train.loc[0,'id']==test.loc[0,'id']:
    check='foo'
    #print(88)
  else:
    check='notfoo'
  train_y=train['cancer'].values
  train_y=to_categorical(train_y,num_classes=2)
  while 1: 
    y=list()
    
    ordered_position=list()
    plan0=list()
    b=list()
    for i in range(1,len(train)):
      slices=list()
      patient=list()
      position=list()
      patient_images=list()
      patient=[o.key for o in storage.Objects('msadata',prefix='test2/'+str(train.loc[i,'id'])+'/',delimiter='')]
      for j in range(len(patient)):
        bucket = storage.Bucket('msadata')
        obj = bucket.object(patient[j])
        dicom_file=obj.read_stream()
        df=dicom.read_file(BytesIO(dicom_file))
        position.append(df.SliceLocation)
      sor=sorted(position)
      ran=[sor.index(x) for x in position]
      o=0
      v=[0 for x in range(len(patient))]
      for x in ran:
        #print(x)
        v[x]=patient[o]
        o=o+1
      patient=v
     # print(position)
      for j in range(len(patient)):
        bucket = storage.Bucket('msadata')
        obj = bucket.object(patient[j])
        dicom_file=obj.read_stream()
        df=dicom.read_file(BytesIO(dicom_file))
        ordered_poistion=df.ImageOrientationPatient
        pixel_Array=df.pixel_array
        pixel_Array=pixel_Array*ordered_poistion[0]+pixel_Array*ordered_poistion[1]+pixel_Array*ordered_poistion[2]
        val=filters.threshold_otsu(pixel_Array)
        mask=pixel_Array>val
        pixel_Array=mask*pixel_Array
        slices.append(pixel_Array)
        if j>0:
            #print(np.abs(np.array([0,0,df.SliceLocation],dtype=np.float)-previous))
            df.SliceThickness=list(np.abs(np.array([df.SliceLocation,0,0],dtype=np.float)-previous))
            #print(df.SliceThickness)
        else:
            #print(np.abs([0,0,df.SliceLocation]))
            df.SliceThickness=list(np.abs([df.SliceLocation,0,0]))
            #print(df.SliceThickness)
        previous=np.array([df.SliceLocation,0,0],dtype=np.float)
      threed_array=np.stack([x for x in slices],axis=0)
      #print(threed_array.shape)
      adjust=(np.array([df.PixelSpacing[0]]*3,dtype=np.float)+np.array(df.SliceThickness,dtype=np.float))/np.array([1,1,1])
      #print(adjust)
      #threed_array=zoom(threed_array,np.round(adjust*threed_array.shape)/threed_array.shape)
      #print(threed_array.shape)
      threed_array=np.expand_dims(threed_array,axis=0)
     # print(threed_array.shape)
      if threed_array.shape[1]>=150:
        threed_array=threed_array[:,:150,:,:]
      else:
        temp=threed_array
        m=threed_array.shape[1]
        threed_array=np.zeros((1,150,512,512))
        threed_array[0,:m,:,:]=temp
      if check=='foo':
        #print('test')
        yield(threed_array)
      else:
        #print('train')
        #print(i)
        #print(np.expand_dims(train_y[i],0))
        yield(threed_array,np.expand_dims(train_y[i],0))
        #model.save('my_model.h5')         
model.fit_generator(array_generator(train),validation_data=array_generator(valid),validation_steps=len(valid),use_multiprocessing=False,steps_per_epoch=len(train),epochs=2)


Epoch 1/2
16
115
90
94
1
131
111
122
46
120
5
77
23
45
53
66
12
74
82
41
50
14
110
15
26
103
132
109
9
76
87
31
19
97
37
81
71
124
40
72
113
133
54
22
28
102
91
96
4
105
130
8
6
69
29
49
99
58
123
125
11
64
116
55
65
47
121
93
25
34
2
128
30
127
95
98
114
117
61
3
51
107
85
10
57
100
101
63
67
78
92
79
17
44
33
83
42
88
89
36
108
48
68
0
60
18
70
35
106
59
84
24
7
21
32
38
126
73
27
52
104
129
20
62
56
13
86
39
119
80
112
75
43
118
train
1
[[1. 0.]]
1/4 [======>.......................] - ETA: 26:44 - loss: nan - acc: 1.000029
79
94
10
0
101
30
106
5
11
86
82
93
71
7
63
108
40
32
59
28
91
69
1
58
64
75
84
33
25
17
110
112
14
15
13
47
38
23
107
76
45
52
55
53
87
90
3
22
97
19
83
62
102
74
42
80
70
105
12
39
100
9
18
60
77
35
27
73
68
41
56
95
96
66
2
4
78
20
65
92
36
21
43
26
54
8
67
16
98
99
61
24
103
104
48
44
109
49
81
113
88
72
89
85
34
50
57
46
37
31
111
6
51
train
2
[[1. 0.]]
2/4 [==============>...............] - ETA: 16:28 - loss: nan - acc: 1.000033
22
75
128
72
88
9
121
89
123


In [8]:
model.evaluate_generator(array_generator(train),steps=len(train))

16
115
90
94
1
131
111
122
46
120
5
77
23
45
53
66
12
74
82
41
50
14
110
15
26
103
132
109
9
76
87
31
19
97
37
81
71
124
40
72
113
133
54
22
28
102
91
96
4
105
130
8
6
69
29
49
99
58
123
125
11
64
116
55
65
47
121
93
25
34
2
128
30
127
95
98
114
117
61
3
51
107
85
10
57
100
101
63
67
78
92
79
17
44
33
83
42
88
89
36
108
48
68
0
60
18
70
35
106
59
84
24
7
21
32
38
126
73
27
52
104
129
20
62
56
13
86
39
119
80
112
75
43
118
train
1
[[1. 0.]]
29
79
94
10
0
101
30
106
5
11
86
82
93
71
7
63
108
40
32
59
28
91
69
1
58
64
75
84
33
25
17
110
112
14
15
13
47
38
23
107
76
45
52
55
53
87
90
3
22
97
19
83
62
102
74
42
80
70
105
12
39
100
9
18
60
77
35
27
73
68
41
56
95
96
66
2
4
78
20
65
92
36
21
43
26
54
8
67
16
98
99
61
24
103
104
48
44
109
49
81
113
88
72
89
85
34
50
57
46
37
31
111
6
51
train
2
[[1. 0.]]
33
22
75
128
72
88
9
121
89
123
83
61
15
81
90
24
68
118
11
105
48
94
92
70
112
62
28
20
130
6
8
65
113
120
35
53
73
77
27
64
50
59
111
108
47
30
114
125
98
54
74
3
79
106
5
26
41
25
38
4
18
9

[nan, 1.0]

In [ ]:
train_y=train['cancer'].values
train_y=to_categorical(train_y,num_classes=2)

In [10]:
model.predict_generator(array_generator(test),steps=len(test))

105
62
22
11
78
24
28
23
1
48
95
60
52
37
87
84
25
108
135
71
88
5
89
128
113
58
110
43
57
8
124
117
120
36
76
123
85
131
41
65
27
18
15
127
91
39
67
133
136
40
96
80
0
66
125
55
12
20
53
50
130
129
63
21
46
107
82
93
16
118
109
119
122
102
104
14
116
74
30
103
97
54
81
134
99
4
2
19
6
115
73
126
44
31
64
114
7
92
49
29
13
77
69
45
47
83
61
90
111
121
132
51
10
100
42
75
94
101
34
26
106
32
79
9
70
38
3
68
86
112
72
33
56
35
59
98
17
204
162
29
60
18
27
151
28
175
98
210
187
228
202
82
225
70
180
23
215
81
58
181
52
101
41
211
76
136
182
88
20
184
168
61
118
112
6
113
183
170
234
240
153
1
87
242
120
30
140
77
236
152
51
166
146
40
37
145
21
212
188
62
178
110
123
206
214
148
169
193
203
80
69
157
17
164
16
132
233
220
90
109
243
74
64
200
239
33
246
217
128
86
224
237
79
241
49
174
102
103
19
83
93
150
190
185
186
130
84
36
44
46
227
43
135
245
115
232
45
106
191
209
171
139
155
198
9
99
38
141
65
154
15
78
231
72
208
137
116
95
3
31
42
134
122
159
147
230
131
219
53
75
158
199
205
12

array([[nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)

In [ ]:
y=list()
plan=list()
plan2=list()
plan0=list()
b=list()
ans=0

for i in range(1,len(train)):
  #print(i)
  slices=list()
  #print(i)
  patientimages=list()
  a=list()
  #print(i)
  a=[o.key for o in storage.Objects('msadata',prefix='test2/'+str(train.loc[i,'id'])+'/',delimiter='')]
  print(len(a))
  for j in range(len(a)):
     #dicom_file=None
    b="gs://msadata/"+patient[j]
    %gcs read --o $b --variable dicom_file
    print(b)
    #dcmfile=dicom.read_file(BytesIO(dicom_file))
    plan.append(dicom.read_file(BytesIO(dicom_file)).ImagePositionPatient)
    print(plan)
  

In [ ]:

def array_generator(train):
      train_y=train['cancer'].values
      train_y=to_categorical(train_y,num_classes=2)
      y=list()
      plan=list()
      plan2=list()
      plan0=list()
      ans=0
      b='gbvdv'
      for i in range(1,len(train)):
      #if i==1:
       # b='test2/2ce2565b9839d8812cf0f9ad80371d3f/0103ce513ca5144e00a907dd7cf4fd64.dcm'
        #%gcs read --o "gs://msadata/$b" --variable dicom_filess
      #print(i)
            slices=list()
      #print(i)
            patientimages=list()
            a=list()
      #print(i)
            a=[o.key for o in storage.Objects('msadata',prefix='test2/'+str(train.loc[i,'id'])+'/',delimiter='')]
            for j in range(0,len(a)):
        #dicom_file=None
                  #b='bikvh'
                  b='gs://msadata/'+patient[j]
                  #print(b)
                  print(type(b))
                  #%gcs read --o '$b' --variable dicom_file
                  bucket = storage.Bucket('msadata')
                  obj = bucket.object(patient[j])
                  dicom_file=obj.read_stream()
                  #print(b)
        #print(dicom_file)
        #dcmfile=dicom.read_file(BytesIO(dicom_file))
                  plan.append(dicom.read_file(BytesIO(dicom_file)).ImagePositionPatient)
        #y.append(b)
                  print(plan)
            sor=sorted(plan,key=lambda x:x[2])
      #print(sor)
            ran=[sor.index(x) for x in plan]
      #print(ran)
            y=[y[x] for x in ran]
      #print(y)
            a=y
      #print(a)
            for j in range(len(y)):
                   b=patient[j]
         #print(b)
                   %gcs read --o "gs://msadata/$b" --variable dicom_file
model.fit_generator(array_generator(train),validation_data=array_generator(valid),validation_steps=len(valid),use_multiprocessing=False,steps_per_epoch=len(train),epochs=2)

In [ ]:
np.expand_dims(train_y[9],0)

In [ ]:
print((train))

In [11]:
train

,id,cancer
0,2ce2565b9839d8812cf0f9ad80371d3f,0.0
1,00cba091fa4ad62cc3200a657aeb957e,0.0
2,65dd5d9d5b894e5f0ac3bd6c1384c46a,0.0
3,0a0c32c9e08cc2ea76a71649de56be6d,0.0
